In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

## Connect to EXACT

In [4]:
configuration = Configuration()
configuration.username = 'marzahl' #'exact'
configuration.password = '###' #'exact'
configuration.host = "https://exact.cs.fau.de" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

## Download Rect annotation

In [5]:
annotation_types_points = {}
annotation_types_ref_rect = None

for product in product_api.list_products(name="References").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id, name="BoundingBox").results:
        annotation_types_ref_rect = annotation_type
            
for product in product_api.list_products(name="Reference_Points").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id).results:
        annotation_types_points[annotation_type.name] = annotation_type

annotation_types_points

{'L0': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 384,
  'name': 'L0',
  'node_count': 0,
  'product': 104,
  'sort_order': 0,
  'vector_type': 1},
 'L1': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 385,
  'name': 'L1',
  'node_count': 0,
  'product': 104,
  'sort_order': 1,
  'vector_type': 1},
 'L2': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': False,
  'enable_concealed': False,
  'id': 386,
  'name': 'L2',
  'node_count': 0,
  'product': 104,
  'sort_order': 2,
  'vector_type': 1},
 'L3': {'area_hit_test': True,
  'closed': True,
  'color_code': '#FF0000',
  'default_height': 50,
  'default_width': 50,
  'enable_blurred': Fa

In [6]:
annotation_types_ref_rect

{'area_hit_test': False,
 'closed': True,
 'color_code': '#FF0000',
 'default_height': 5000,
 'default_width': 5000,
 'enable_blurred': False,
 'enable_concealed': False,
 'id': 412,
 'name': 'BoundingBox',
 'node_count': 0,
 'product': 105,
 'sort_order': 0,
 'vector_type': 1}

In [7]:
annotations = []

image_set = image_sets_api.list_image_sets(name="CCMCT_Aperio", expand="product_set,product_set.annotationtype_set").results[0]


for image_id in image_set.images:
    
    image = images_api.retrieve_image(id=image_id)
    
    for anno in annotations_api.list_annotations(image=image_id, annotation_type=annotation_types_ref_rect.id, # BoundBox ID
                                                 deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:
        
        annotations.append([image.id, image.name, anno.id, anno.vector, anno.unique_identifier, anno.annotation_type])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "id", "vector", "unique_identifier", "annotation_type"])
annotations.head()

,image_id,image_name,id,vector,unique_identifier,annotation_type
0,10416,A_CCMCT_518711B_1.svs,2754780,"{'x1': 2286, 'x2': 105081, 'y1': 2658, 'y2': 8...",4dc7b8b1-9f22-459c-aea8-5b6a4c3ea550,412
1,10415,A_CCMCT_380609B_1.svs,2754776,"{'x1': 3384, 'x2': 122851, 'y1': 3932, 'y2': 8...",9ed713ff-9533-40bd-bbcf-dd63c9c4b674,412
2,10401,A_CCMCT_183715A_1.svs,2754779,"{'x1': 2795, 'x2': 74520, 'y1': 1848, 'y2': 64...",d630bd0a-94af-4ebb-b44b-a4a548b391d1,412
3,10399,A_CCMCT_29609B_1.svs,2754778,"{'x1': 15671, 'x2': 61564, 'y1': 3806, 'y2': 6...",94970f48-cebb-4874-9211-38e408592a13,412
4,10398,A_CCMCT_22108_1.svs,2754777,"{'x1': 3516, 'x2': 97828, 'y1': 3382, 'y2': 85...",a41f016f-02fb-40da-af8d-b99fb1c8686d,412


## Load GT and find closest point

In [5]:
Path("../../CCMCT/CCMCT.csv").exists()

True

In [10]:
df = pd.read_csv("../../CCMCT/CCMCT.csv", delimiter=";", names=["image_name", "grade", "vector"])
df["vector"] = [json.loads(vector)  for vector in df["vector"]]

df["x1"] = [vector['x1'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y1"] = [vector['y1'] if type(vector) is dict else 0 for vector in df["vector"]]

df["x2"] = [vector['x2'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y2"] = [vector['y2'] if type(vector) is dict else 0 for vector in df["vector"]]

df["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(df["x1"], df["x2"])]
df["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(df["y1"], df["y2"])]

df["anno_width"] = [x2-x1 for x1, x2 in zip(df["x1"], df["x2"])]
df["anno_height"]= [y2-y1 for y1, y2 in zip(df["y1"], df["y2"])]

df.head()

,image_name,grade,vector,x1,y1,x2,y2,center_x,center_y,anno_width,anno_height
0,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 57204, 'y1': 28229, 'x2': 57254, 'y2': ...",57204,28229,57254,28279,57229.0,28254.0,50,50
1,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 57895, 'y1': 37440, 'x2': 57945, 'y2': ...",57895,37440,57945,37490,57920.0,37465.0,50,50
2,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 54810, 'y1': 37131, 'x2': 54860, 'y2': ...",54810,37131,54860,37181,54835.0,37156.0,50,50
3,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 54575, 'y1': 34537, 'x2': 54625, 'y2': ...",54575,34537,54625,34587,54600.0,34562.0,50,50
4,A_CCMCT_29609B_1.svs,mitosis,"{'x1': 47883, 'y1': 28677, 'x2': 47933, 'y2': ...",47883,28677,47933,28727,47908.0,28702.0,50,50


In [17]:
temp

,image_name,grade,vector,x1,y1,x2,y2,center_x,center_y,anno_width,anno_height


In [18]:
num_lines = 5
patch_size = 25
search_point_radius = 0.15

registration_points = []
for id, bounding_box in annotations.iterrows():
    
    df_gt = df[df["image_name"].str.contains(bounding_box.image_name.split("_")[-2])]
    
    box_width = bounding_box.vector["x2"] - bounding_box.vector["x1"]
    box_height = bounding_box.vector["y2"] - bounding_box.vector["y1"]
    
    step_size_x = box_width // num_lines
    step_size_y = box_height // num_lines
    
    gt_search_point_radius = int(step_size_x * search_point_radius)
    
    point_index = 0
    for x in range(bounding_box.vector["x1"] + step_size_x //2, bounding_box.vector["x2"], step_size_x):
        for y in range(bounding_box.vector["y1"] + step_size_y //2, bounding_box.vector["y2"], step_size_y):
            
            ref_point = annotation_types_points[f"L{point_index}"]
            
            point = None
            # find closest annotation in range
            temp = df_gt[(df_gt["center_x"] > x - gt_search_point_radius) & 
                          (df_gt["center_x"] < x + gt_search_point_radius) & 
                          (df_gt["center_y"] > y - gt_search_point_radius) & 
                          (df_gt["center_y"] < y + gt_search_point_radius)].copy()
            
            if len(temp) > 0:
                temp["distance"] = [abs(cx - x) + abs(cy - y) for cx, cy in zip(temp["center_x"], temp["center_y"])]
                temp = temp.sort_values(by=["distance"], ascending=[True])

                for uuid, gt_anno in temp.iterrows():

                    vector = {"x1": gt_anno.vector["x1"], "y1": gt_anno.vector["y1"], 
                              "x2": gt_anno.vector["x2"], "y2": gt_anno.vector["y2"]}


                    point = [bounding_box.image_id, vector, point_index, ref_point.name, ref_point.id]
                    break
            
            if point is None: # No ref point found
                
                vector = {"x1": x - patch_size, "y1": y - patch_size, 
                          "x2": x + patch_size, "y2": y + patch_size}
                
                point = [bounding_box.image_id, vector, point_index, ref_point.name, ref_point.id]
                
            registration_points.append(point)
                
            point_index += 1
    
registration_points = pd.DataFrame(registration_points, 
                                   columns=["image_id", "vector", "point_index", "point_name", "point_id"])
registration_points

,image_id,vector,point_index,point_name,point_id
0,10416,"{'x1': 12540, 'y1': 10554, 'x2': 12590, 'y2': ...",0,L0,384
1,10416,"{'x1': 15549, 'y1': 25719, 'x2': 15599, 'y2': ...",1,L1,385
2,10416,"{'x1': 13378, 'y1': 42978, 'x2': 13428, 'y2': ...",2,L2,386
3,10416,"{'x1': 12126, 'y1': 58187, 'x2': 12176, 'y2': ...",3,L3,387
4,10416,"{'x1': 13407, 'y1': 71707, 'x2': 13457, 'y2': ...",4,L4,388
...,...,...,...,...,...
120,10398,"{'x1': 88370, 'y1': 11524, 'x2': 88420, 'y2': ...",20,L20,407
121,10398,"{'x1': 88370, 'y1': 27859, 'x2': 88420, 'y2': ...",21,L21,408
122,10398,"{'x1': 88370, 'y1': 44194, 'x2': 88420, 'y2': ...",22,L22,409
123,10398,"{'x1': 88370, 'y1': 60529, 'x2': 88420, 'y2': ...",23,L23,410


## Upload ref Points to EXACT

In [19]:
annotations_to_upload = []

for id, bounding_box in tqdm(registration_points.iterrows()):
    
    
    exact_annos =  annotations_api.list_annotations(image=bounding_box.image_id, annotation_type=bounding_box.point_id).results
    
    if len(exact_annos) == 0:
        anno = Annotation(annotation_type=bounding_box.point_id, 
                          vector=bounding_box.vector, 
                          image=bounding_box.image_id)

        annotations_to_upload.append(anno)

    else:
        prinf(f"ImageId: {bounding_box.image_id}, AnnotationType: {bounding_box.point_id} combination allready exists!")

125it [00:24,  5.09it/s]


In [20]:
annotations_to_upload[:3]

[{'annotation_type': 384,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10416,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': None,
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 12540, 'x2': 12590, 'y1': 10554, 'y2': 10604},
  'verified_by_user': None},
 {'annotation_type': 385,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10416,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': None,
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 15549, 'x2': 15599, 'y1': 25719, 'y2': 25769},
  'verified_by_user': None},
 {'annotation_type': 386,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id':

## Check that no error occurred and afterwards upload annotations

In [21]:
#annotations_api.create_annotation(body=annotations_to_upload)

[{'annotation_type': 384,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2755281,
  'image': 10416,
  'last_edit_time': datetime.datetime(2021, 1, 10, 16, 56, 54, 572334),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 10, 16, 56, 54, 572309),
  'unique_identifier': 'ca3fde15-30ed-45bd-bd58-00aadf29eba9',
  'uploaded_media_files': [],
  'user': 1,
  'vector': {'x1': 12540, 'x2': 12590, 'y1': 10554, 'y2': 10604},
  'verified_by_user': 'False'},
 {'annotation_type': 385,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2755282,
  'image': 10416,
  'last_edit_time': datetime.datetime(2021, 1, 10, 16, 56, 54, 572395),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 10, 16, 56, 54, 572382),
  'unique_identifier': '0acc9849-e7aa-4657-8055-bb7e8b4a31e0',
  'uploaded_media_files':